In [1]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('Mirai_pcap.pcap.tsv', sep='\t')
ft = pd.read_csv('dataset_bd.csv')
df.columns = df.columns.str.replace('.', '_')
sources = df.ip_src.unique()
destinations = df.ip_dst.unique()
print(sources)
print(destinations)

for i, j in zip(sources, destinations):
    if i == j:
        print(i)
    else:
        print("n: " + str(i))
        
oneOneZero = df.loc[df['ip_dst'] == '192.168.2.113']
#print(oneOneZero)
#oneOneZero.to_csv('oOZ.csv')

print(oneOneZero.shape)

print(df.shape)

['192.168.2.108' '192.168.2.1' nan '192.168.2.101' '192.168.2.110'
 '192.168.2.115' '192.168.2.112' '192.168.2.113' '192.168.2.109'
 '192.168.2.117' '192.168.2.119' '192.168.2.104' '192.168.2.126'
 '192.168.2.105' '0.0.0.0' '192.168.2.196' '192.168.2.120' '192.168.2.118'
 '192.168.2.103' '192.168.2.111' '192.168.2.121' '192.168.2.122'
 '192.168.2.107']
['52.24.43.67' '52.25.66.250' '192.168.2.108' nan '192.168.2.110'
 '192.168.2.101' '192.168.2.1' '192.168.2.115' '114.114.114.114'
 '50.19.254.134' '122.248.234.207' '46.137.188.54' '122.226.84.253'
 '61.188.37.216' '120.24.59.150' '114.215.137.159' '104.199.156.58'
 '192.168.2.113' '239.255.255.250' '192.168.2.112' '74.125.31.99'
 '192.168.2.109' '8.8.8.8' '112.74.102.136' '54.154.5.26' '54.149.224.179'
 '192.168.2.126' '210.61.248.232' '42.99.254.162' '203.69.81.91'
 '61.220.62.219' '119.29.29.29' '255.255.255.255' '224.0.0.22'
 '192.168.1.252' '112.74.124.169' '192.168.2.119' '133.100.9.2'
 '192.168.2.196' '139.78.100.163' '131.107.1.

C:\Users\17lch\AppData\Local\Temp/ipykernel_21524/3814262777.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace('.', '_')


In [13]:
#sorted = df.sort_values(by='ip_src')
#print(sorted)
#sorted.to_csv('sorted.csv')
oneZeroEight = df.loc[df['ip_src'] == '192.168.2.113']

# <blank> ms is 1 second
s = 10 #number of packets in burst
g = 0.1 #size of gap in seconds between packets in burst
t = 20 #threshold of seconds of when to give up completing current burst
p = 1 #period of delayed burst manipuation in seconds

timeSorted = oneZeroEight.sort_values(by='frame_time_epoch')
startTime = timeSorted.iloc[0, 0]
endTime = timeSorted.iloc[-1, 0]
currentTime = startTime

allGreater = timeSorted.loc[timeSorted['frame_time_epoch'] > currentTime]
greater = allGreater.iloc[0, 0]
    
lastPossibleGreater = allGreater.loc[allGreater['frame_time_epoch'] < greater + t]

#display(timeSorted)
#display(lastPossibleGreater.index)
#display(lastPossibleGreater.head(s))
#burst = lastPossibleGreater.head(s)
#display(len(burst.index))
#display(burst)
#display(burst.iloc[1])
#display(burst.loc[9])
#display(burst.index.to_list())
#posIndex = burst.index.to_list()
#display(posIndex[9])
#burst.iloc[9, 0] = burst.iloc[9, 0] + g
#display(burst.iloc[9, 0])

#display(lastPossibleGreater.index.get_loc(9))

firt = True

while currentTime < endTime:
    currentTime = currentTime + p #wait for period
    
    allGreater = timeSorted.loc[timeSorted['frame_time_epoch'] > currentTime]
    
    if allGreater.empty:
        print('out of data to process')
        break
    
    greater = allGreater.iloc[0, 0]
    lastPossibleGreater = allGreater.loc[allGreater['frame_time_epoch'] < greater + t]
    burst = lastPossibleGreater.head(s)
    maxBurst = len(burst.index)
    
    burst.iloc[maxBurst - 1, 0] = burst.iloc[maxBurst - 1, 0] + g
    
    posIndex = burst.index.to_list()
    currentPosIndex = posIndex[maxBurst - 1]
    
    oneZeroEight.loc[currentPosIndex, 0] = burst.iloc[maxBurst - 1, 0]
    
    valu = burst.iloc[maxBurst - 1, 0]
    backwards = 2
    
    if firt == True:
        display(burst)
    
    while backwards <= maxBurst:
        
        burst.iloc[maxBurst - backwards, 0] = valu - (g * (backwards - 1))
        
        currentPosIndex = posIndex[maxBurst - backwards]
        trueIndex = oneZeroEight.index.get_loc(currentPosIndex)
        #if firt == True:
            #display(oneZeroEight.iloc[trueIndex, 0])
        oneZeroEight.iloc[trueIndex, 0] = burst.iloc[maxBurst - backwards, 0]
        #if firt == True:
            #display(burst.iloc[maxBurst - backwards, 0])
            #display(oneZeroEight.iloc[trueIndex, 0])
            #print('===================')
        
        backwards = backwards + 1
        
    firt = False
    #survey_df.iloc[0].replace(to_replace=120, value = 130)
    #burst.loc[maxBurst - 1, 0] = burst.iloc[maxBurst - 1, 0] + g
    
oneZeroEight.to_csv('oneOneThree.tsv', index=False, sep='\t')
oneZeroEight.to_csv('oneOneThree.csv', index=False)
    




C:\Users\17lch\anaconda3\lib\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)


,frame_time_epoch,frame_len,eth_src,eth_dst,ip_src,ip_dst,tcp_srcport,tcp_dstport,udp_srcport,udp_dstport,icmp_type,icmp_code,arp_opcode,arp_src_hw_mac,arp_src_proto_ipv4,arp_dst_hw_mac,arp_dst_proto_ipv4,ipv6_src,ipv6_dst
57,1.540446e+09,82,34:c3:d2:e0:fc:c9,4c:09:d4:c6:12:7b,192.168.2.113,50.19.254.134,NaN,NaN,50861.0,10240.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,1.540446e+09,82,34:c3:d2:e0:fc:c9,4c:09:d4:c6:12:7b,192.168.2.113,122.248.234.207,NaN,NaN,50861.0,10240.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,1.540446e+09,82,34:c3:d2:e0:fc:c9,4c:09:d4:c6:12:7b,192.168.2.113,46.137.188.54,NaN,NaN,50861.0,10240.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,1.540446e+09,82,34:c3:d2:e0:fc:c9,4c:09:d4:c6:12:7b,192.168.2.113,122.226.84.253,NaN,NaN,50861.0,10240.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,1.540446e+09,82,34:c3:d2:e0:fc:c9,4c:09:d4:c6:12:7b,192.168.2.113,61.188.37.216,NaN,NaN,50861.0,10240.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,1.540446e+09,82,34:c3:d2:e0:fc:c9,4c:09:d4:c6:12:7b,192.168.2.113,120.24.59.150,NaN,NaN,50861.0,10240.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,1.540446e+09,82,34:c3:d2:e0:fc:c9,4c:09:d4:c6:12:7b,192.168.2.113,114.215.137.159,NaN,NaN,50861.0,10240.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,1.540446e+09,82,34:c3:d2:e0:fc:c9,4c:09:d4:c6:12:7b,192.168.2.113,104.199.156.58,NaN,NaN,50861.0,10240.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,1.540446e+09,82,34:c3:d2:e0:fc:c9,4c:09:d4:c6:12:7b,192.168.2.113,50.19.254.134,NaN,NaN,50861.0,10240.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74,1.540446e+09,82,34:c3:d2:e0:fc:c9,4c:09:d4:c6:12:7b,192.168.2.113,122.248.234.207,NaN,NaN,50861.0,10240.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


C:\Users\17lch\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


out of data to process


In [5]:
oneZeroEight = df.loc[df['ip_src'] == '192.168.2.108']
oneZeroEight.to_csv('oneZeroEight.tsv', index=False, sep='\t')
oneZeroEight.to_csv('oneZeroEight.csv', index=False)

print(oneZeroEight.shape)

(51747, 19)


In [70]:
one = df.loc[df['ip_src'] == '192.168.2.1']
one.to_csv('one.tsv', index=False, sep='\t')
one.to_csv('one.csv', index=False)

In [10]:
oneZeroOne = df.loc[df['Source'] == '192.168.2.101']
oneZeroOne.to_csv('oneZeroOne.tsv', index=False, sep='\t')
oneZeroOne.to_csv('oneZeroOne.csv')

KeyError: 'Source'

In [11]:
oneOneZero = df.loc[df['ip_src'] == '192.168.2.110']
oneOneZero.to_csv('oneOneZero.tsv', index=False, sep='\t')
oneOneZero.to_csv('oneOneZero.csv')
print(oneOneZero.shape)

(23554, 19)


In [82]:
oneOneFive = df.loc[df['ip_src'] == '192.168.2.115']
oneOneFive.to_csv('oneOneFive.tsv', index=False, sep='\t')
oneOneFive.to_csv('oneOneFive.csv')

In [83]:
oneOneTwo = df.loc[df['ip_src'] == '192.168.2.112']
oneOneTwo.to_csv('oneOneTwo.tsv', index=False, sep='\t')
oneOneTwo.to_csv('oneOneTwo.csv')

In [84]:
oneOneThree = df.loc[df['ip_src'] == '192.168.2.113']
oneOneThree.to_csv('oneOneThree.tsv', index=False, sep='\t')
oneOneThree.to_csv('oneOneThree.csv')

In [85]:
oneOneTwo = df.loc[df['ip_src'] == '192.168.2.109']
oneOneTwo.to_csv('oneZeroNine.tsv', index=False, sep='\t')
oneOneTwo.to_csv('oneZeroNine.csv')

In [86]:
oneOneTwo = df.loc[df['ip_src'] == '192.168.2.117']
oneOneTwo.to_csv('oneOneSeven.tsv', index=False, sep='\t')
oneOneTwo.to_csv('oneOneSeven.csv')

In [87]:
oneOneTwo = df.loc[df['ip_src'] == '192.168.2.119']
oneOneTwo.to_csv('oneOneNine.tsv', index=False, sep='\t')
oneOneTwo.to_csv('oneOneNine.csv')

In [88]:
oneOneTwo = df.loc[df['ip_src'] == '192.168.2.104']
oneOneTwo.to_csv('oneZeroFour.tsv', index=False, sep='\t')
oneOneTwo.to_csv('oneZeroFour.csv')

In [89]:
oneOneTwo = df.loc[df['ip_src'] == '192.168.2.126']
oneOneTwo.to_csv('oneTwoSix.tsv', index=False, sep='\t')
oneOneTwo.to_csv('oneTwoSix.csv')

In [90]:
oneOneTwo = df.loc[df['ip_src'] == '192.168.2.105']
oneOneTwo.to_csv('oneZeroFive.tsv', index=False, sep='\t')
oneOneTwo.to_csv('oneZeroFive.csv')

In [91]:
oneOneTwo = df.loc[df['ip_src'] == '192.168.2.196']
oneOneTwo.to_csv('oneNineSix.tsv', index=False, sep='\t')
oneOneTwo.to_csv('oneNineSix.csv')

In [92]:
oneOneTwo = df.loc[df['ip_src'] == '192.168.2.120']
oneOneTwo.to_csv('oneTwoZero.tsv', index=False, sep='\t')
oneOneTwo.to_csv('oneTwoZero.csv')

In [93]:
oneOneTwo = df.loc[df['ip_src'] == '192.168.2.118']
oneOneTwo.to_csv('oneOneEight.tsv', index=False, sep='\t')
oneOneTwo.to_csv('oneOneEight.csv')

In [94]:
oneOneTwo = df.loc[df['ip_src'] == '192.168.2.103']
oneOneTwo.to_csv('oneZeroThree.tsv', index=False, sep='\t')
oneOneTwo.to_csv('oneZeroThree.csv')

In [95]:
oneOneTwo = df.loc[df['ip_src'] == '192.168.2.111']
oneOneTwo.to_csv('oneOneOne.tsv', index=False, sep='\t')
oneOneTwo.to_csv('oneOneOne.csv')

In [96]:
oneOneTwo = df.loc[df['ip_src'] == '192.168.2.121']
oneOneTwo.to_csv('oneTwoOne.tsv', index=False, sep='\t')
oneOneTwo.to_csv('oneTwoOne.csv')

In [98]:
oneOneTwo = df.loc[df['ip_src'] == '192.168.2.122']
oneOneTwo.to_csv('oneTwoTwo.tsv', index=False, sep='\t')
oneOneTwo.to_csv('oneTwoTwo.csv')

oneOneTwoTime = oneOneTwo.sort_values(by='frame_time_epoch')
print(oneOneTwoTime)



for i in range(len(oneOneTwoTime)):
    if df.loc[i, "frame_time_epoch"]

        frame_time_epoch  frame_len            eth_src            eth_dst  \
74752       1.540449e+09         75  50:c7:bf:1c:9c:c6  4c:09:d4:c6:12:7b   
74753       1.540449e+09         62  50:c7:bf:1c:9c:c6  4c:09:d4:c6:12:7b   
74766       1.540449e+09         80  50:c7:bf:1c:9c:c6  4c:09:d4:c6:12:7b   
75059       1.540449e+09         75  50:c7:bf:1c:9c:c6  4c:09:d4:c6:12:7b   
75063       1.540449e+09         80  50:c7:bf:1c:9c:c6  4c:09:d4:c6:12:7b   
...                  ...        ...                ...                ...   
757950      1.540453e+09         75  50:c7:bf:1c:9c:c6  4c:09:d4:c6:12:7b   
760183      1.540454e+09         80  50:c7:bf:1c:9c:c6  4c:09:d4:c6:12:7b   
760547      1.540454e+09         75  50:c7:bf:1c:9c:c6  4c:09:d4:c6:12:7b   
762604      1.540454e+09         80  50:c7:bf:1c:9c:c6  4c:09:d4:c6:12:7b   
763016      1.540454e+09         75  50:c7:bf:1c:9c:c6  4c:09:d4:c6:12:7b   

               ip_src       ip_dst  tcp_srcport  tcp_dstport  udp_srcport  